### Downloading Train Dataset Images

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
import requests
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from google.colab import drive

In [3]:
# --- 1. Mount Google Drive ---
try:
    drive.mount('/content/drive')
    print("Google Drive mounted successfully!")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")
    exit()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully!


In [ ]:
# --- 2. Configuration ---
CSV_FILE_PATH = '/content/drive/My Drive/student_resource/dataset/train.csv'
IMAGE_DOWNLOAD_FOLDER = '/content/drive/My Drive/student_resource/dataset/train_images'
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

In [ ]:
# --- 3. Modified Image Downloader Functions ---

def download_image(sample_id, image_link, save_folder):
    """
    Downloads a single image and saves it using its sample_id as the filename.
    This version will OVERWRITE any existing file with the same name.
    """
    if not isinstance(image_link, str) or not image_link.startswith(('http', 'https')):
        return ('error', f"Invalid URL for sample_id {sample_id}: {image_link}")

    try:
        file_extension = Path(image_link.split('?')[0]).suffix
        if not file_extension:
             file_extension = '.jpg'

        filename = f"{sample_id}{file_extension}"
        image_save_path = os.path.join(save_folder, filename)

        # --- MODIFICATION ---
        # The check for os.path.exists() has been REMOVED.
        # The script will now proceed to download and save the file,
        # overwriting it if it already exists.

        response = requests.get(image_link, headers=HEADERS, timeout=15, stream=True)
        response.raise_for_status()

        with open(image_save_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

        return ('success', image_link)

    except requests.exceptions.RequestException as e:
        return ('error', f"Failed to download {image_link} (sample_id: {sample_id}): {e}")
    except Exception as ex:
        return ('error', f"An unexpected error occurred for {image_link} (sample_id: {sample_id}): {ex}")

def download_images_threaded(image_data, download_folder, max_workers=50):
    """
    Downloads images from a list of (sample_id, image_link) tuples.
    """
    if not os.path.exists(download_folder):
        print(f"Creating directory: {download_folder}")
        os.makedirs(download_folder)

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(download_image, sample_id, link, download_folder): link for sample_id, link in image_data}

        success_count = 0
        failed_downloads = []

        for future in tqdm(as_completed(futures), total=len(image_data), desc="Downloading Images"):
            status, message = future.result()
            if status == 'success':
                success_count += 1
            else: # 'error'
                failed_downloads.append(message)

    # --- Final Summary Report ---
    print("\n" + "="*50)
    print("Download Complete!")
    print(f"✅ Successfully downloaded/overwritten: {success_count}")
    print(f"❌ Failed to download: {len(failed_downloads)}")
    print("="*50)

    if failed_downloads:
        print("\n--- Failed URLs ---")
        for fail in failed_downloads[:10]:
            print(fail)
        if len(failed_downloads) > 10:
            print(f"... and {len(failed_downloads) - 10} more.")

In [ ]:
# --- 4. Main Execution ---
if __name__ == "__main__":
    try:
        print(f"Reading CSV file from: {CSV_FILE_PATH}")
        df = pd.read_csv(CSV_FILE_PATH)

        if 'image_link' not in df.columns or 'sample_id' not in df.columns:
            print("Error: 'sample_id' or 'image_link' column not found.")
        else:
            df.dropna(subset=['image_link'], inplace=True)
            image_data_to_download = list(zip(df['sample_id'], df['image_link']))

            print(f"Found {len(image_data_to_download)} images to download.")
            download_images_threaded(image_data_to_download, IMAGE_DOWNLOAD_FOLDER)

    except FileNotFoundError:
        print(f"Error: The file was not found at {CSV_FILE_PATH}. Please check the path.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [ ]:
import os

# ❗️ IMPORTANT: Replace this with the actual path to your folder in Google Drive
folder_path = '/content/drive/My Drive/student_resource/dataset/train_images'

# List of common image file extensions
image_extensions = ('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff', '.webp')

image_count = 0

try:
  # Get a list of all files in the specified folder
  files_in_folder = os.listdir(folder_path)

  # Loop through each file and check its extension
  for file_name in files_in_folder:
    if file_name.lower().endswith(image_extensions):
      image_count += 1

  print(f"✅ Found {image_count} images in the folder.")
  print(f"📂 Path: {folder_path}")

except FileNotFoundError:
  print(f"❌ ERROR: The path '{folder_path}' was not found.")
  print("Please double-check your folder path and try again.")

✅ Found 74999 images in the folder.
📂 Path: /content/drive/My Drive/student_resource/dataset/train_images


### Downloading Test Data Images

In [ ]:
import os
import pandas as pd
import requests
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from google.colab import drive

In [ ]:
# --- 1. Mount Google Drive ---
try:
    drive.mount('/content/drive')
    print("Google Drive mounted successfully!")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")
    exit()

Mounted at /content/drive
Google Drive mounted successfully!


In [ ]:
# --- 2. Configuration (MODIFIED FOR TEST DATA) ---
# Updated path to point to the test.csv file
CSV_FILE_PATH = '/content/drive/My Drive/student_resource/dataset/test.csv'
# Updated path to save images in a new 'test_images' folder
IMAGE_DOWNLOAD_FOLDER = '/content/drive/My Drive/student_resource/dataset/test_images'
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

In [ ]:
# --- 3. Image Downloader Functions (No changes needed here) ---

def download_image(sample_id, image_link, save_folder):
    """
    Downloads a single image and saves it using its sample_id as the filename.
    This version will OVERWRITE any existing file with the same name.
    """
    if not isinstance(image_link, str) or not image_link.startswith(('http', 'https')):
        return ('error', f"Invalid URL for sample_id {sample_id}: {image_link}")

    try:
        file_extension = Path(image_link.split('?')[0]).suffix
        if not file_extension:
             file_extension = '.jpg'

        filename = f"{sample_id}{file_extension}"
        image_save_path = os.path.join(save_folder, filename)

        response = requests.get(image_link, headers=HEADERS, timeout=15, stream=True)
        response.raise_for_status()

        with open(image_save_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

        return ('success', image_link)

    except requests.exceptions.RequestException as e:
        return ('error', f"Failed to download {image_link} (sample_id: {sample_id}): {e}")
    except Exception as ex:
        return ('error', f"An unexpected error occurred for {image_link} (sample_id: {sample_id}): {ex}")

def download_images_threaded(image_data, download_folder, max_workers=50):
    """
    Downloads images from a list of (sample_id, image_link) tuples.
    """
    if not os.path.exists(download_folder):
        print(f"Creating directory: {download_folder}")
        os.makedirs(download_folder)

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(download_image, sample_id, link, download_folder): link for sample_id, link in image_data}

        success_count = 0
        failed_downloads = []

        for future in tqdm(as_completed(futures), total=len(image_data), desc="Downloading Test Images"):
            status, message = future.result()
            if status == 'success':
                success_count += 1
            else: # 'error'
                failed_downloads.append(message)

    # --- Final Summary Report ---
    print("\n" + "="*50)
    print("Download Complete!")
    print(f"✅ Successfully downloaded/overwritten: {success_count}")
    print(f"❌ Failed to download: {len(failed_downloads)}")
    print("="*50)

    if failed_downloads:
        print("\n--- Failed URLs ---")
        for fail in failed_downloads[:10]:
            print(fail)
        if len(failed_downloads) > 10:
            print(f"... and {len(failed_downloads) - 10} more.")

# --- 4. Main Execution (Reads from test.csv) ---
if __name__ == "__main__":
    try:
        print(f"Reading CSV file from: {CSV_FILE_PATH}")
        df = pd.read_csv(CSV_FILE_PATH)

        if 'image_link' not in df.columns or 'sample_id' not in df.columns:
            print("Error: 'sample_id' or 'image_link' column not found.")
        else:
            df.dropna(subset=['image_link'], inplace=True)
            image_data_to_download = list(zip(df['sample_id'], df['image_link']))

            print(f"Found {len(image_data_to_download)} test images to download.")
            download_images_threaded(image_data_to_download, IMAGE_DOWNLOAD_FOLDER)

    except FileNotFoundError:
        print(f"Error: The file was not found at {CSV_FILE_PATH}. Please check the path.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

Reading CSV file from: /content/drive/My Drive/student_resource/dataset/test.csv
Found 75000 test images to download.



Download Complete!
✅ Successfully downloaded/overwritten: 74999
❌ Failed to download: 1

--- Failed URLs ---
Failed to download https://m.media-amazon.com/images/I/813CjSgHj0S.jpg (sample_id: 286800): 404 Client Error: Not Found for url: https://m.media-amazon.com/images/I/813CjSgHj0S.jpg


In [ ]:
import os

# ❗️ IMPORTANT: Replace this with the actual path to your folder in Google Drive
folder_path = '/content/drive/My Drive/student_resource/dataset/test_images'

# List of common image file extensions
image_extensions = ('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff', '.webp')

image_count = 0

try:
  # Get a list of all files in the specified folder
  files_in_folder = os.listdir(folder_path)

  # Loop through each file and check its extension
  for file_name in files_in_folder:
    if file_name.lower().endswith(image_extensions):
      image_count += 1

  print(f"✅ Found {image_count} images in the folder.")
  print(f"📂 Path: {folder_path}")

except FileNotFoundError:
  print(f"❌ ERROR: The path '{folder_path}' was not found.")
  print("Please double-check your folder path and try again.")

✅ Found 74999 images in the folder.
📂 Path: /content/drive/My Drive/student_resource/dataset/test_images


# Training Part

### ResNet-18 and TF-IDF

In [4]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import resnet18, ResNet18_Weights
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import numpy as np

In [5]:
# ====================================================
# CONFIG
# ====================================================
TRAIN_CSV = "/content/drive/My Drive/student_resource/dataset/train.csv"
TRAIN_IMG_DIR = "/content/drive/My Drive/student_resource/dataset/train_images"
TEST_IMG_DIR = "/content/drive/My Drive/student_resource/dataset/test_images"
OUTPUT_SUBMISSION = "/content/drive/My Drive/student_resource/dataset/submission.csv"

BATCH_SIZE = 32
EPOCHS = 2   # try small first, can increase later
LR = 2e-4
TEXT_DIM = 512
IMG_DIM = 512  # output dim for resnet18


In [6]:
# ====================================================
# LOAD DATA
# ====================================================
df = pd.read_csv(TRAIN_CSV)
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)
print(f"Train samples: {len(train_df)}, Validation samples: {len(val_df)}")

Train samples: 67500, Validation samples: 7500


In [7]:
# ====================================================
# TF-IDF VECTOR
# ====================================================
vectorizer = TfidfVectorizer(max_features=TEXT_DIM)
vectorizer.fit(train_df['catalog_content'].astype(str).tolist())

def text_to_tensor(text_list):
    X = vectorizer.transform(text_list).toarray()
    return torch.tensor(X, dtype=torch.float)

In [8]:
# ====================================================
# DATASET CLASS
# ====================================================
class AMCPriceDataset(Dataset):
    def __init__(self, df, img_dir, transform, is_test=False):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        sample_id = str(row["sample_id"])
        img_path = os.path.join(self.img_dir, f"{sample_id}.jpg")

        # Handle missing images
        if not os.path.exists(img_path):
            image = torch.zeros(3, 160, 160)
        else:
            try:
                image = Image.open(img_path).convert("RGB")
                image = self.transform(image)
            except:
                image = torch.zeros(3, 160, 160)

        text_emb = text_to_tensor([str(row["catalog_content"] or "")])[0]

        item = {
            "image": image,
            "text_emb": text_emb,
            "sample_id": sample_id
        }
        if not self.is_test:
            item["price"] = torch.tensor(row["price"], dtype=torch.float)
        return item

In [9]:
# ====================================================
# IMAGE TRANSFORMS
# ====================================================
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [10]:
# ====================================================
# DATALOADERS
# ====================================================
train_ds = AMCPriceDataset(train_df, TRAIN_IMG_DIR, transform)
val_ds = AMCPriceDataset(val_df, TRAIN_IMG_DIR, transform)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [11]:
# ====================================================
# MODEL
# ====================================================
class MultiModalRegressor(nn.Module):
    def __init__(self, text_dim=TEXT_DIM, img_dim=IMG_DIM):
        super().__init__()
        self.img_encoder = resnet18(weights=ResNet18_Weights.DEFAULT)
        self.img_encoder.fc = nn.Identity()
        for param in self.img_encoder.parameters():
            param.requires_grad = False  # freeze CNN

        self.fc = nn.Sequential(
            nn.Linear(text_dim + img_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 1)
        )

    def forward(self, text_emb, image):
        img_feat = self.img_encoder(image)
        fused = torch.cat([text_emb, img_feat], dim=1)
        return self.fc(fused).squeeze(1)

In [12]:
# ====================================================
# TRAINING SETUP
# ====================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultiModalRegressor().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
criterion = nn.MSELoss()
scaler = torch.cuda.amp.GradScaler()

def mae(preds, targets):
    return torch.mean(torch.abs(preds - targets))

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 81.9MB/s]
/tmp/ipython-input-1880077353.py:8: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
/usr/local/lib/python3.12/dist-packages/torch/amp/grad_scaler.py:136: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


In [ ]:
# ====================================================
# TRAINING LOOP
# ====================================================
best_val_mae = float("inf")

for epoch in range(EPOCHS):
    # ---- Train ----
    model.train()
    total_train_loss, total_train_mae = 0, 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Train]"):
        optimizer.zero_grad()
        text_emb = batch["text_emb"].to(device)
        image = batch["image"].to(device)
        price = batch["price"].to(device)

        with torch.cuda.amp.autocast():
            preds = model(text_emb, image)
            loss = criterion(preds, price)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_train_loss += loss.item()
        total_train_mae += mae(preds, price).item()

    avg_train_loss = total_train_loss / len(train_loader)
    avg_train_mae = total_train_mae / len(train_loader)

    # ---- Validation ----
    model.eval()
    total_val_loss, total_val_mae = 0, 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Val]"):
            text_emb = batch["text_emb"].to(device)
            image = batch["image"].to(device)
            price = batch["price"].to(device)
            preds = model(text_emb, image)
            loss = criterion(preds, price)
            total_val_loss += loss.item()
            total_val_mae += mae(preds, price).item()

    avg_val_loss = total_val_loss / len(val_loader)
    avg_val_mae = total_val_mae / len(val_loader)

    print(f"Epoch {epoch+1}: Train Loss={avg_train_loss:.4f}, Train MAE={avg_train_mae:.4f}, "
          f"Val Loss={avg_val_loss:.4f}, Val MAE={avg_val_mae:.4f}")

    if avg_val_mae < best_val_mae:
        best_val_mae = avg_val_mae
        torch.save(model.state_dict(), "/content/drive/My Drive/student_resource/dataset/best_model.pt")
        print(f"✅ Saved best model (Val MAE={best_val_mae:.4f})")

Epoch 1/2 [Train]:   0%|          | 0/2110 [00:00<?, ?it/s]/tmp/ipython-input-1419317186.py:16: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1/2 [Val]: 100%|██████████| 235/235 [08:17<00:00,  2.12s/it]


Epoch 1: Train Loss=1022.7983, Train MAE=16.9473, Val Loss=916.3157, Val MAE=17.6432
✅ Saved best model (Val MAE=17.6432)


Epoch 2/2 [Val]: 100%|██████████| 235/235 [08:07<00:00,  2.07s/it]


Epoch 2: Train Loss=995.7673, Train MAE=16.6392, Val Loss=899.4309, Val MAE=16.7063
✅ Saved best model (Val MAE=16.7063)


In [ ]:
# ====================================================
# INFERENCE / SUBMISSION
# ====================================================
test_csv_path = "/content/drive/My Drive/student_resource/dataset/test.csv"
if os.path.exists(test_csv_path):
    test_df = pd.read_csv(test_csv_path)
    test_ds = AMCPriceDataset(test_df, TEST_IMG_DIR, transform, is_test=True)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

    model.load_state_dict(torch.load("/content/drive/My Drive/student_resource/dataset/best_model.pt", map_location=device))
    model.eval()

    preds_list, ids_list = [], []
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Predicting Test"):
            text_emb = batch["text_emb"].to(device)
            image = batch["image"].to(device)
            preds = model(text_emb, image)
            preds_list.extend(preds.cpu().numpy().tolist())
            ids_list.extend(batch["sample_id"])

    submission = pd.DataFrame({
        "sample_id": ids_list,
        "price": np.clip(preds_list, 0, None)
    })
    submission.to_csv(OUTPUT_SUBMISSION, index=False)
    print("✅ Submission saved to:", OUTPUT_SUBMISSION)
else:
    print("⚠️ No test.csv found; skipped submission generation.")

Predicting Test:   0%|          | 0/2344 [00:00<?, ?it/s]

### Sentence Transformer + EfficientNet + Manual Features + a LightGBM/XGBoost/CatBoost Ensemble

In [4]:
# Run this once to force reinstall
import os
if os.path.exists('/content/.libs_installed'):
    os.remove('/content/.libs_installed')
    print("✅ Marker file removed. Now run the installation cell again.")

✅ Marker file removed. Now run the installation cell again.


In [1]:
# ====================================================
# SECTION 1: SETUP & INSTALLATIONS (Fixed)
# ====================================================
import os

# Check if we need to install
if not os.path.exists('/content/.libs_installed'):
    print("🚀 Installing necessary libraries with compatible versions...")

    # Uninstall problematic packages
    !pip uninstall -y sentence-transformers catboost -q

    # Install with correct versions (catboost needs to be compiled with current numpy)
    !pip install sentence-transformers>=2.3.0 lightgbm==4.1.0 xgboost==2.0.3 -q
    !pip install catboost --no-cache-dir -q

    # Mark as installed
    with open('/content/.libs_installed', 'w') as f:
        f.write('done')

    print("✅ Installation complete!")
    print("🔄 Restarting runtime automatically...")

    # Auto-restart runtime
    os.kill(os.getpid(), 9)
else:
    print("✅ Libraries already installed, proceeding with imports...")

✅ Libraries already installed, proceeding with imports...


In [2]:
# Imports
import re
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from PIL import Image
from sentence_transformers import SentenceTransformer
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from tqdm.auto import tqdm
from google.colab import drive

# Mount Google Drive
try:
    drive.mount('/content/drive')
    print("✅ Google Drive mounted successfully!")
except Exception as e:
    print(f"❌ Error mounting Google Drive: {e}")

print("✅ All libraries imported successfully!")
print(f"📦 NumPy version: {np.__version__}")
print(f"📦 CatBoost version: {cb.__version__}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive mounted successfully!
✅ All libraries imported successfully!
📦 NumPy version: 2.0.2
📦 CatBoost version: 1.2.8


In [3]:
# ====================================================
# SECTION 2: CONFIGURATION
# ====================================================
print("\n📝 Setting up configuration...")
DRIVE_BASE_PATH = "/content/drive/My Drive/student_resource/dataset"
TRAIN_CSV = os.path.join(DRIVE_BASE_PATH, "train.csv")
TEST_CSV = os.path.join(DRIVE_BASE_PATH, "test.csv")
TRAIN_IMG_DIR = os.path.join(DRIVE_BASE_PATH, "train_images")
TEST_IMG_DIR = os.path.join(DRIVE_BASE_PATH, "test_images")
OUTPUT_SUBMISSION = os.path.join(DRIVE_BASE_PATH, "submission_ensemble.csv")

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
TEXT_MODEL_NAME = 'all-MiniLM-L6-v2'
BATCH_SIZE = 64
print(f"Using device: {DEVICE}")


📝 Setting up configuration...
Using device: cpu


In [4]:
# ====================================================
# SECTION 3: FEATURE EXTRACTION & ENGINEERING
# ====================================================

def extract_text_embeddings(df, model_name):
    print(f"\nEmbedding text with '{model_name}'...")
    model = SentenceTransformer(model_name, device=DEVICE)
    sentences = df['catalog_content'].astype(str).tolist()
    embeddings = model.encode(sentences, show_progress_bar=True, batch_size=BATCH_SIZE)
    return embeddings

class ImgDataset(Dataset):
    def __init__(self, df, img_dir):
        self.df = df
        self.img_dir = img_dir
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, f"{row['sample_id']}.jpg")
        try:
            image = Image.open(img_path).convert("RGB")
            return self.transform(image)
        except (FileNotFoundError, IOError):
            return torch.zeros(3, 224, 224)

def extract_image_embeddings(df, img_dir):
    print(f"\nExtracting image features with EfficientNet-B0...")
    model = efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)
    model.classifier = nn.Identity()
    model.to(DEVICE)
    model.eval()
    dataset = ImgDataset(df, img_dir)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    all_features = []
    with torch.no_grad():
        for images in tqdm(dataloader, desc="Processing images"):
            images = images.to(DEVICE)
            features = model(images)
            all_features.append(features.cpu().numpy())
    return np.vstack(all_features)

def create_manual_features(df):
    print("\nEngineering manual text features...")
    # IPQ Extraction
    def extract_ipq(text):
        if not isinstance(text, str): return 1
        match = re.search(r'(?:Item Pack Quantity|IPQ)\s*[:\-]?\s*(\d+)', text, re.IGNORECASE)
        return int(match.group(1)) if match else 1
    df['ipq'] = df['catalog_content'].apply(extract_ipq)

    # Other Text-based Features
    df['text_len'] = df['catalog_content'].astype(str).apply(len)
    df['num_digits'] = df['catalog_content'].astype(str).apply(lambda x: sum(c.isdigit() for c in x))
    df['num_caps'] = df['catalog_content'].astype(str).apply(lambda x: sum(c.isupper() for c in x))

    return df[['ipq', 'text_len', 'num_digits', 'num_caps']].values


In [5]:
# ====================================================
# SECTION 4: DATA PREPARATION
# ====================================================
print("\n🔄 Loading and preparing data...")
train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)

# Extract all features
train_text_feats = extract_text_embeddings(train_df, TEXT_MODEL_NAME)
test_text_feats = extract_text_embeddings(test_df, TEXT_MODEL_NAME)

train_img_feats = extract_image_embeddings(train_df, TRAIN_IMG_DIR)
test_img_feats = extract_image_embeddings(test_df, TEST_IMG_DIR)

train_manual_feats = create_manual_features(train_df)
test_manual_feats = create_manual_features(test_df)

# Combine into final training matrices
X_train = np.hstack([train_text_feats, train_img_feats, train_manual_feats])
X_test = np.hstack([test_text_feats, test_img_feats, test_manual_feats])
y_train = np.log1p(train_df['price']) # Use log-transformed price

print(f"\n✅ Feature extraction and combination complete.")
print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")



🔄 Loading and preparing data...

Embedding text with 'all-MiniLM-L6-v2'...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

100%|██████████| 20.5M/20.5M [00:00<00:00, 120MB/s] 


Processing images:   0%|          | 0/1172 [00:00<?, ?it/s]


Extracting image features with EfficientNet-B0...


Processing images:   0%|          | 0/1172 [00:00<?, ?it/s]


Engineering manual text features...

Engineering manual text features...

✅ Feature extraction and combination complete.
Training data shape: (75000, 1668)
Test data shape: (75000, 1668)


In [ ]:
# ==============================================================================
# SECTION 5 & 6 (COMBINED): K-FOLD VALIDATION, METRICS, PLOTS & SUBMISSION
# ==============================================================================
# This script should be run after Section 4 (Data Preparation) is complete.

# --- Required additional imports ---
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1. Define the SMAPE Metric Function ---
def smape(y_true, y_pred):
    """
    Calculates the Symmetric Mean Absolute Percentage Error (SMAPE).
    """
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    # Add a small epsilon to the denominator to avoid division by zero
    epsilon = 1e-8
    return np.mean(numerator / (denominator + epsilon)) * 100

# --- 2. Set up K-Fold Cross-Validation ---
NFOLDS = 5
kf = KFold(n_splits=NFOLDS, shuffle=True, random_state=42)

# Arrays to store out-of-fold (OOF) predictions for the training set
oof_preds_lgbm = np.zeros(X_train.shape[0])
oof_preds_xgb = np.zeros(X_train.shape[0])
oof_preds_cat = np.zeros(X_train.shape[0])

# Arrays to store predictions for the test set (averaged over folds)
test_preds_lgbm = np.zeros(X_test.shape[0])
test_preds_xgb = np.zeros(X_test.shape[0])
test_preds_cat = np.zeros(X_test.shape[0])

# --- 3. Train Models with K-Fold ---
print("\n🧠 Training Ensemble of Models with K-Fold Cross-Validation...")

# --- LightGBM ---
lgbm_params = {'objective': 'regression_l1', 'metric': 'rmse', 'n_estimators': 2000, 'learning_rate': 0.01,
               'feature_fraction': 0.8, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'lambda_l1': 0.1,
               'lambda_l2': 0.1, 'num_leaves': 31, 'verbose': -1, 'n_jobs': -1, 'seed': 42}

for fold, (train_index, val_index) in enumerate(kf.split(X_train, y_train)):
    print(f"--- LightGBM: Fold {fold+1}/{NFOLDS} ---")
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    model = lgb.LGBMRegressor(**lgbm_params)
    model.fit(X_train_fold, y_train_fold,
              eval_set=[(X_val_fold, y_val_fold)],
              callbacks=[lgb.early_stopping(100, verbose=False)])

    oof_preds_lgbm[val_index] = model.predict(X_val_fold)
    test_preds_lgbm += model.predict(X_test) / NFOLDS

# --- XGBoost ---
xgb_params = {'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'seed': 42, 'n_estimators': 1000,
              'learning_rate': 0.02, 'colsample_bytree': 0.8, 'subsample': 0.8, 'n_jobs': -1}
for fold, (train_index, val_index) in enumerate(kf.split(X_train, y_train)):
    print(f"--- XGBoost: Fold {fold+1}/{NFOLDS} ---")
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    model = xgb.XGBRegressor(**xgb_params)
    model.fit(X_train_fold, y_train_fold, eval_set=[(X_val_fold, y_val_fold)], early_stopping_rounds=50, verbose=False)
    oof_preds_xgb[val_index] = model.predict(X_val_fold)
    test_preds_xgb += model.predict(X_test) / NFOLDS

# --- CatBoost ---
cat_params = {'iterations': 2000, 'learning_rate': 0.02, 'eval_metric': 'RMSE', 'random_seed': 42,
              'verbose': 0, 'early_stopping_rounds': 50}
for fold, (train_index, val_index) in enumerate(kf.split(X_train, y_train)):
    print(f"--- CatBoost: Fold {fold+1}/{NFOLDS} ---")
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    model = cb.CatBoostRegressor(**cat_params)
    model.fit(X_train_fold, y_train_fold, eval_set=[(X_val_fold, y_val_fold)])
    oof_preds_cat[val_index] = model.predict(X_val_fold)
    test_preds_cat += model.predict(X_test) / NFOLDS

# --- 4. Calculate Validation Metrics ---
print("\n" + "="*50)
print("📊 CALCULATING OUT-OF-FOLD (OOF) VALIDATION SCORES")
print("="*50)
# Get the true prices from the original dataframe
true_prices = train_df['price'].values

# Calculate SMAPE for each individual model
oof_price_lgbm = np.expm1(oof_preds_lgbm)
smape_lgbm = smape(true_prices, oof_price_lgbm)
print(f"LightGBM OOF SMAPE Score: {smape_lgbm:.4f}%")

oof_price_xgb = np.expm1(oof_preds_xgb)
smape_xgb = smape(true_prices, oof_price_xgb)
print(f"XGBoost OOF SMAPE Score: {smape_xgb:.4f}%")

oof_price_cat = np.expm1(oof_preds_cat)
smape_cat = smape(true_prices, oof_price_cat)
print(f"CatBoost OOF SMAPE Score: {smape_cat:.4f}%")

# Calculate SMAPE for the final ensemble
oof_ensemble_preds = (oof_preds_lgbm + oof_preds_xgb + oof_preds_cat) / 3.0
oof_price_ensemble = np.expm1(oof_ensemble_preds)
smape_ensemble = smape(true_prices, oof_price_ensemble)
print("-" * 50)
print(f"🏆 Final Ensemble OOF SMAPE Score: {smape_ensemble:.4f}%")
print("-" * 50)

# --- 5. Generate Plots for the Report ---
print("\n📈 GENERATING PLOTS FOR APPENDIX")

# Plot 1: Prediction vs. Actual Price (for the final ensemble)
plt.figure(figsize=(10, 10))
# Use a subset for plotting to avoid overplotting if the dataset is huge
plot_subset_indices = np.random.choice(len(true_prices), size=min(10000, len(true_prices)), replace=False)
sns.scatterplot(x=true_prices[plot_subset_indices], y=oof_price_ensemble[plot_subset_indices], alpha=0.4, edgecolor=None)
plt.plot([min(true_prices), max(true_prices)], [min(true_prices), max(true_prices)], color='red', linestyle='--', linewidth=2, label='Perfect Prediction')
plt.title('Actual Price vs. OOF Predicted Price (Ensemble)', fontsize=16)
plt.xlabel('Actual Price ($) - Log Scale', fontsize=12)
plt.ylabel('Predicted Price ($) - Log Scale', fontsize=12)
plt.xscale('log')
plt.yscale('log')
plt.grid(True, which="both", ls="--")
plt.legend()
plt.show()

# Plot 2: Distribution of Price vs. Log-Transformed Price
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
sns.histplot(train_df['price'], bins=50, kde=True, ax=axes[0], color='skyblue')
axes[0].set_title('Distribution of Original Price (Skewed)')
axes[0].set_xlabel('Price ($)')
sns.histplot(y_train, bins=50, kde=True, ax=axes[1], color='salmon')
axes[1].set_title('Distribution of Log-Transformed Price (More Normal)')
axes[1].set_xlabel('log(1 + Price)')
plt.suptitle('Price Distribution Before and After Log Transformation', fontsize=18)
plt.show()


🧠 Training Ensemble of Models with K-Fold Cross-Validation...
--- LightGBM: Fold 1/5 ---


/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


--- LightGBM: Fold 2/5 ---


/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [ ]:
# --- 6. Final Submission Generation ---
print("\n📤 CREATING FINAL SUBMISSION FILE")

# Average the test predictions from all three models
ensemble_test_preds = (test_preds_lgbm + test_preds_xgb + test_preds_cat) / 3.0

# Convert predictions back from log scale and ensure non-negativity
final_predictions = np.expm1(ensemble_test_preds)
final_predictions = np.clip(final_predictions, 0, None)

# Create submission DataFrame
submission_df = pd.DataFrame({
    "sample_id": test_df["sample_id"],
    "price": final_predictions
})

submission_df.to_csv(OUTPUT_SUBMISSION, index=False)
print(f"🎉 Success! Submission file created at: {OUTPUT_SUBMISSION}")
print("\nTop 5 predictions:")
print(submission_df.head())